In [1]:
Round=145

import warnings
warnings.filterwarnings('ignore')
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.externals import joblib
import os.path
import numerapi

from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from catboost import CatBoostClassifier
import random, timeit, string

import warnings; warnings.simplefilter('ignore')

/home/samit/anaconda3/envs/env1/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
napi = numerapi.NumerAPI(verbosity="info")
if napi.check_new_round():
    print("new round has started within the last 24hours!")
else:
    print("no new round within the last 24 hours")

def load_data(ro):
    ros=str(ro)
    if os.path.isfile('../datasets/'+ros+'-train.csv'):
        return pd.read_csv('../datasets/'+ros+'-train.csv'),pd.read_csv('../datasets/'+ros+'-tour.csv')
    else:
        print ('Checking Round')
        cc=napi.get_current_round()
        if cc==ro:
            print ('Downloading files for Round ',ro)
            napi.download_current_dataset(unzip=True)
        else:
            print ('Older round or round not started')


new round has started within the last 24hours!


In [4]:
train,tour=load_data(Round)
ship = tour[tour['data_type']=='validation']

In [5]:
features=[f for f in list(train) if "feature" in f]
print (features)
targets=[f.split('_')[1] for f in list(train) if "target" in f]
print (targets)
target=targets[0]
target

['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6', 'feature7', 'feature8', 'feature9', 'feature10', 'feature11', 'feature12', 'feature13', 'feature14', 'feature15', 'feature16', 'feature17', 'feature18', 'feature19', 'feature20', 'feature21', 'feature22', 'feature23', 'feature24', 'feature25', 'feature26', 'feature27', 'feature28', 'feature29', 'feature30', 'feature31', 'feature32', 'feature33', 'feature34', 'feature35', 'feature36', 'feature37', 'feature38', 'feature39', 'feature40', 'feature41', 'feature42', 'feature43', 'feature44', 'feature45', 'feature46', 'feature47', 'feature48', 'feature49', 'feature50']
['bernie', 'elizabeth', 'jordan', 'ken', 'charles', 'frank', 'hillary']


'bernie'

In [6]:
def PCA_transformed(n):
    pca = PCA(n_components=n)
    pca.fit(train[features])
    return pca

In [7]:
def rounder(x,level=3):
    if x>=.7:
        return .7
    if x<.3:
        return .3
    mu=10**level
    x=x*mu
    if x>(mu/2):
        return (int(x)+1)/float(mu)
    return int(x)/float(mu)
ron = np.vectorize(rounder)

def loss(y,yp):
    return metrics.log_loss(y,ron(yp))

def exo(cut,pca = 24,onTrain=0):
    pca = PCA_transformed(pca)
    X=pca.transform(cut[features])
    if onTrain==2:
        return X
    y=cut['target_'+target]
    if onTrain:
        return X,y
    yp=cut['pred']
    return X,y,yp,loss(y,yp)

# Caluclates consistency score and log loss
def eraLoss(frame,pred,pca):
    eras=frame['era'].unique()
    frame['pred']=pred
    li=[]
    for era in eras:
        X,y,yp,l=exo(frame[frame['era']==era], pca=pca)
        li.append(l)
    li=np.array(li)
    X,y,yp,l=exo(frame,pca)
    c=len(li[li<.693])*100/len(li)
    print ('Loss:', l, '- at ',c,'%')
    print ('-- -- '*8)
    print()
    return l, c

In [8]:
def rr(x,y):
    if type(x)==type(1) and type(y)==type(1):
        x=int(x)
        y=int(y)
        return random.randint(x,y+1)
    return random.uniform(x,y)

def rrr(a,b): #test b4 use
    if type(a) == int and type(b) == int:
        c= int((b-a)/4)
    else:
        c= (b-a)/4
    if random.choice([0,1]):
        return rr(a,a+c)                   # search in first  quadrant
    return rr(b-c,b)                       # search in fourth quadrant

In [9]:
def param_gen(emap):                       # emap is a dictionary which has range of values [a,b] for each parameter
    m={}
    for i in emap:
        m[i]=rrr(emap[i][0],emap[i][1])    # for each paramter in the dict provide the upper and lower limit values i.e a and b
    return m                               # return a dictionary with single value for each parameter.  

In [10]:
def mutate(a,b):
    ones=[a,b]
    if type(a) == int and type(b) == int:
        muls=[int((a+b)/2),int((2*a+b)/3),int((2*b+a)/3)]
    else:
        muls=[(a+b)/2,(2*a+b)/3,(2*b+a)/3]
#     print (muls)
#     print (ones+muls+muls)
    return random.choice(ones+muls+muls)

def params_child(p1,p2):
#     print (p1,p2)
    m={}
    for i in p1:                         
        if p1[i]==p2[i]:
            m[i]=p1[i]                          # When parameters are same for both the parents keep them in child else mutate
        else:                                   
            m[i]=mutate(p1[i],p2[i])
    if m==p1:
        return 0
    if m==p2:
        return 0
    return m
def rands(n):
    return ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(n))

In [11]:
def getModel(ty='xgb',params={}):
    if ty=='xgb':
        copy = dict(params)                # making a copy of dict, to delete key safely
        del copy["pca"]
        copy['nthread']=-1
        return XGBClassifier(**copy)
    if ty=='cb':
        copy = dict(params)
        del copy["pca"]
        copy['thread_count']= 4
        copy['verbose']= False
        return CatBoostClassifier(**copy)
    if ty=='lgbm':
        copy = dict(params)
        del copy["pca"]
        return LGBMClassifier(**copy)    

In [12]:
maps={
    'xgb':{'params':{'max_depth':[1,15],'n_estimators':[10,2000],'learning_rate':[.01,.9],'gamma':[0,100], 'min_child_weight':[1,20],
           'max_delta_step':[1,1000], 'subsample':[.05,1.0],'colsample_bytree':[.05,1.0], 
           'colsample_bylevel':[.05,1.0], 'reg_alpha':[0,1000], 'reg_lambda':[0,1000], 'pca': [5,35]},
           'hookups':4,
           'children':3,
           'asuras':2,
           'survivors':5
          },
    'lgbm':{'params':{'num_leaves':[2,80],'max_depth':[1,20],'n_estimators':[10,2000],'learning_rate':[.01,.9],
           'subsample':[.05,1.0],'colsample_bytree':[.05,1.0],'reg_alpha':[0,1000],'reg_lambda':[0,1000],'pca': [5,35]},
           'hookups':10,
           'children':4,
           'asuras':6,
           'survivors':9
          },
    'cb':{'params':{'depth':[1,15],
          'iterations':[250,750],
          'learning_rate':[0.03,0.3], 
          'l2_leaf_reg':[3,100],
          'border_count':[32,200],'pca': [5,35]
          },
          'hookups':4,
           'children':4,
           'asuras':4,
          'survivors':5
         }
}

In [13]:
# this populate the performance of all the classifiers in genome dataframe
def simulate():
    for i,j in genome.iterrows():
        if not j['ran']:
            clock1=timeit.timeit()
            print (j['name'])
            clf=getModel(j['type'],j['params'])
            X,y = exo(train,j['params']["pca"],1)
            Xt,yt=exo(ship,j['params']["pca"],1)
            clf.fit(X,y)
            yp=clf.predict_proba(Xt)[:,1]
            l,c=eraLoss(ship,yp,j['params']["pca"])
            clock2=timeit.timeit()
            genome.loc[i,'log_loss']=l
            genome.loc[i,'consistency']=c
            genome.loc[i,'delay']=round((clock2-clock1)*1000,2)
            genome.loc[i,'ran']=1

In [14]:
def genesis():
    for m in maps:
        for i in range(maps[m]['asuras']):
            genome.loc[len(genome)]=[m+'-gen0-orig-'+rands(4),m,param_gen(maps[m]['params']),.8,0,0,0]

In [15]:
def reproduce(gen):
    for m in maps:
        for h in range(maps[m]['hookups']):
            parents = genome[genome['type']==m].sample(n=2)
            for c in range(maps[m]['children']):
                child=params_child(parents.iloc[0]['params'],parents.iloc[1]['params'])
                if child:
                    genome.loc[len(genome)]=[m+'-gen'+str(gen)+'-h'+str(h)+'-c'+str(c)+'-'+rands(4),m,child,0.8,0,0,0]


In [16]:
def war(genome):
    count=0
    survivors=[]
    for m in maps:
        if count:
            sids=genome[genome['type']==m].sort_values(['consistency','log_loss'], ascending=[False,True]).head(maps[m]['survivors']).index
            survivors=survivors.append(sids)
        else:
            survivors=genome[genome['type']==m].sort_values(['consistency','log_loss'], ascending=[False,True]).head(maps[m]['survivors']).index
        count=count+1
    return genome.iloc[survivors].reset_index(drop=True)

In [17]:
# genome=pd.DataFrame(columns=['name','type','params','log_loss','consistency','delay','ran'])

genome=pd.read_csv('../genome.csv')
genome['params']=genome['params'].apply(eval)

In [18]:
target

'bernie'

In [19]:

for x in targets:
    target=x
    print (target)
    
    for generation in range(2,5):
        reproduce(generation)
        genesis()
        simulate()
        genome=war(genome)
        genome.to_csv('../genome.csv',index=False)
        genome.to_csv('../genome-gen'+str(generation)+'.csv',index=False)

    # Stacking by kfold split


    train_copy = train.copy()
    tour_copy = tour.copy()


    from sklearn.model_selection import GroupKFold

    gkf = GroupKFold(n_splits=5)
    X,Y = exo (train,1,onTrain=1)   # We just need index so PCA -1 does not matter here
    kfold_split = gkf.split(X, Y, groups=train.era)

    for train_index, test_index in kfold_split:
        print("TRAIN:", train_index, "TEST:", test_index)

        for i,j in genome.iterrows():
            clf=getModel(j['type'],j['params'])
            X,Y = exo(train,j['params']["pca"],1)
            Xt,yt=exo(tour,j['params']["pca"],1)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]
            clf.fit(X_train,y_train)
            train_copy.loc[test_index,"feature_{}_{}".format(j['type'],i)]=clf.predict_proba(X_test)[:,1]

            clf.fit(X,Y)
            tour_copy["feature_{}_{}".format(j['type'],i)]=clf.predict_proba(Xt)[:,1]  




    features_new=[f for f in list(train_copy) if "feature_" in f]
    X_reg = train_copy[features_new]
    y_reg = train_copy['target_'+target]

    stacker = LogisticRegression(max_iter=250).fit(X_reg,y_reg)
    final_preds = stacker.predict_proba(tour_copy[features_new])[:,1]

    results_df = pd.DataFrame(data=final_preds,columns=['probability_'+target])
    joined = pd.DataFrame(tour['id']).join(results_df)
    joined.head()

    sp='../submissions/'+target+"meta_model_stacked_submission.csv"
    joined.to_csv(sp, index=False)

    # Adding public and private keys
    a='6X4PZS5QYKRW5RIN6LK4PY3KGAIQBZJPEW5CPEXNAM6IRBDTDLG4QV3NFWMMJQZI'
    b='WWYHT5NBSLOPKL6VLPNFRZMRG5MHDKPI'
    napi = numerapi.NumerAPI(b, a)

    target

    tim={'bernie':1, 'charles':5, 'elizabeth':2, 'jordan':3, 'ken':4, 'frank':6, 'hillary':7}

    try:
        submission_id = napi.upload_predictions(sp,tournament=tim[target])
    except KeyboardInterrupt:
        raise
    except:
        print ("\n Error Uploading file \n")

bernie
xgb-gen2-h0-c0-M1YQ
Loss: 0.6922856300101943 - at  83.33333333333333 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen2-h0-c1-NMA1
Loss: 0.6924516011896993 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen2-h0-c2-T80Q
Loss: 0.6923197211567805 - at  83.33333333333333 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen2-h1-c0-WDQ7
Loss: 0.692244857301593 - at  83.33333333333333 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen2-h1-c1-I6Z2
Loss: 0.6922834574269103 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen2-h1-c2-G6WO
Loss: 0.6923214839143422 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen2-h2-c0-YPAI
Loss: 0.6923384862262545 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen2-h2-c1-MTOU
Loss: 0.6924155334953912 - at  83.33333333333333 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen2-h3-c0-38HC
Loss: 0.6923559077210885 - at  75.0 %
-- -- -- -- -- -- --

Loss: 0.6925620247390902 - at  58.333333333333336 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

cb-gen0-orig-I89K
Loss: 0.7071126727676761 - at  0.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

cb-gen0-orig-G8NK
Loss: 0.6923224498526782 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen3-h0-c0-J6B1
Loss: 0.6923145333552647 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen3-h0-c1-9LUA
Loss: 0.6923468038433028 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen3-h0-c2-2BRT
Loss: 0.6923981151609133 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen3-h1-c0-R02S
Loss: 0.6923738069927731 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen3-h1-c1-8J5D
Loss: 0.6923465957827107 - at  83.33333333333333 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen3-h1-c2-KLJD
Loss: 0.692383631569333 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen3-h2-c0-15Z9
Loss: 0

Loss: 0.6927197883124198 - at  66.66666666666667 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

lgbm-gen0-orig-MF3V
Loss: 0.6926786920999517 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

lgbm-gen0-orig-IZKO
Loss: 0.7036276625383835 - at  0.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

cb-gen0-orig-F1B2
Loss: 0.6988752520480166 - at  0.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

cb-gen0-orig-9L8W
Loss: 0.6925972192873093 - at  66.66666666666667 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

cb-gen0-orig-9RQR
Loss: 0.6922949741196504 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

cb-gen0-orig-7YN4
Loss: 0.6924784901364887 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen4-h0-c0-MZFF
Loss: 0.6923486724710401 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen4-h0-c1-EQUV
Loss: 0.6923233975560683 - at  83.33333333333333 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen4-h0-c2-65GC

Loss: 0.6927065176585863 - at  66.66666666666667 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

lgbm-gen0-orig-TLEB
Loss: 0.6926434804680265 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

lgbm-gen0-orig-XXMD
Loss: 0.6925900496433285 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

lgbm-gen0-orig-YEG2
Loss: 0.6925696704285327 - at  66.66666666666667 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

lgbm-gen0-orig-OKFC
Loss: 0.6926944005251091 - at  66.66666666666667 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

lgbm-gen0-orig-00VD
Loss: 0.6926542961529577 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

cb-gen0-orig-HW9F
Loss: 0.7426150963615018 - at  0.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

cb-gen0-orig-QF4A
Loss: 0.6939173769254314 - at  25.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

cb-gen0-orig-MBEW
Loss: 0.718068607811796 - at  0.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

cb-gen0-orig-2QHP


2019-02-03 06:38:08,690 INFO numerapi.numerapi: uploading predictions...


...

Loss: 0.6923888153917448 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen4-h1-c0-RVR5
Loss: 0.6922916464124459 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen4-h1-c1-U4SR
Loss: 0.6923386396425814 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen4-h1-c2-ZYU0
Loss: 0.6923536573633116 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen4-h2-c0-HWPP
Loss: 0.6923885987742157 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen4-h2-c1-FYWX
Loss: 0.6923169606615204 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen4-h2-c2-42KH
Loss: 0.6923720241873444 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen4-h3-c0-XAJO
Loss: 0.6923269857884171 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen4-h3-c1-5SC2
Loss: 0.6924051638246639 - at  75.0 %
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 

xgb-gen4-h3-c2-5NMA
Loss: 0.6923741879015546 - a

TRAIN: [  2425   2426   2427 ... 484902 484903 484904] TEST: [     0      1      2 ... 502729 502730 502731]
TRAIN: [     0      1      2 ... 502729 502730 502731] TEST: [  7255   7256   7257 ... 480506 480507 480508]
TRAIN: [     0      1      2 ... 502729 502730 502731] TEST: [  4815   4816   4817 ... 453501 453502 453503]
TRAIN: [     0      1      2 ... 502729 502730 502731] TEST: [  9818   9819   9820 ... 462594 462595 462596]


2019-02-03 15:58:06,137 INFO numerapi.numerapi: uploading predictions...


ken
xgb-gen2-h0-c0-AQAA


KeyboardInterrupt: 

In [ ]:
# stop here